In [161]:
#Computação científica
import numpy as np

#análise de dados
import pandas as pd

#visualização
import matplotlib.pyplot as plt

#machine learning
import sklearn

# feature engineering
from sklearn.impute import SimpleImputer
from feature_engine.imputation import (
    AddMissingIndicator)
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.encoding import  RareLabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from feature_engine.wrappers import SklearnTransformerWrapper


#Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

In [162]:
sklearn.set_config(display='diagram')
sklearn.set_config(transform_output="pandas")

# Introdução

## Motivação


## Objetivos


# Download Dataset

In [163]:
# O dataset está disponível no seguinte URL:
#https://www.kaggle.com/competitions/porto-seguro-safe-driver-prediction/data

path='/home/rodolfo/Insync/rodolfopcruz2@gmail.com/Google Drive/Estudo/Python_Projects/datasets/porto-seguro-safe-driver-prediction/'
train=pd.read_csv(path+'train.csv')
test= pd.read_csv(path+'test.csv')

In [164]:
cat_features=[feature for feature in train.columns if 'cat' in feature]

bin_features=[feature for feature in train.columns if 'bin' in feature]

num_features=[feature for feature in train.columns if 'cat' not in feature 
                                                    and 'bin' not in feature and
                                                    feature!='target' and feature!='id']


In [165]:
y_train=train['target']
train  =train.drop(columns=['target','id'])
test   =test.drop(columns=["id"])

In [166]:
#Nos dados os missing values estão representados por -1
#Substituir -1 por np.nan para facilitar a identificação
train=train.replace(-1,np.nan)

# Pipelines

- Serão criadas duas pipelines de dados:

    - Em uma delas serão tratadas todas as colunas, a seleção das features será feita mais a frente;
    - Na outra a primeira etapa consistirá na remoção de algumas festures, de acordo com os resultados obtidos com a correlação.

## Pipeline para dados com todas as features

In [167]:
#Nos dados os missing values foram substituídos pelo valor -1
#Função para trocar -1 por np .nan

def impute_nan_missing_value(x):
    x=x.replace(-1,np.nan)
    return x

impute_nan_missing_value_transformer=FunctionTransformer(impute_nan_missing_value)

In [168]:
#Função para converter data type para object

def converter_object(x):
    x=x.astype('object')
    return x

converter_object_transformer=FunctionTransformer(converter_object)

### Features Binárias

Pipeline para features binárias:

1) Substituir -1 por np.nan;
2) Converter os valores para object;
3) Substituir valores ausentes pelo mais frequente.

In [169]:
#imputer para substituir missing values
imputer_binary=SimpleImputer(strategy='most_frequent')

In [170]:
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                          ('convert_to_object',converter_object_transformer),
                          ('fill_missing_binary',imputer_binary)])

### Features Numéricas

Pipeline para features numéricas:

1) Os missing values estão representados por -1. Substituir por np.nan;
2) Para as colunas numéricas com muitos missing values será criada um nova coluna para identificar um valor ausente;
3) Os valores ausentes de todas as colunas numéricas serão substituídos pela média;
4) Corrigir a distribuição de algumas colunas;
5) Normalização dos dados das colunas.


In [171]:
missing_threshold=1/100

#numeric features com muitos mussing values
num_features_muitos_na=train[num_features].isna().mean()>missing_threshold
num_features_muitos_na=num_features_muitos_na[num_features_muitos_na].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na=train[num_features].isna().mean()<missing_threshold
num_features_poucos_na=num_features_poucos_na[num_features_poucos_na].index.to_list()

In [172]:
#Criar nova coluna para indicar se existe valor ausente nas colunas numéricas com muitos na

missing_indicator=AddMissingIndicator(variables=num_features_muitos_na)
#A coluna criada tem o mesmo nome da original acrescido da terminação _na

In [173]:
#Substituir os valores ausentes pela média
#Será aplicado em todas as colunas numéricas

num_imputer=SimpleImputer(strategy='mean')


In [174]:
#Aplicar transformação de Yeo Jhonson as seguintes features

features_to_be_transformed=['ps_reg_03',
                            'ps_car_12',
                            'ps_car_13',
                            'ps_car_14',
                            'ps_car_15',
                            'ps_reg_02']

yeo_transformer=YeoJohnsonTransformer(variables=features_to_be_transformed)

In [175]:
# Scaling

scaler = StandardScaler()

In [176]:
pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=num_features))])


### Features Categóricas

In [177]:
cat_features

['ps_ind_02_cat',
 'ps_ind_04_cat',
 'ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_02_cat',
 'ps_car_03_cat',
 'ps_car_04_cat',
 'ps_car_05_cat',
 'ps_car_06_cat',
 'ps_car_07_cat',
 'ps_car_08_cat',
 'ps_car_09_cat',
 'ps_car_10_cat',
 'ps_car_11_cat']

In [178]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na=train[cat_features].isna().mean()>missing_threshold
cat_features_muitos_na=cat_features_muitos_na[cat_features_muitos_na].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na=train[cat_features].isna().mean()<missing_threshold
cat_features_poucos_na=cat_features_poucos_na[cat_features_poucos_na].index.to_list()

In [179]:
#Substituir missing values nas features com muitos valores ausentes
cat_imputer_muitos_na=SimpleImputer(strategy='constant',fill_value='missing')

#Substituir missing values nas features com poucos valores ausentes
cat_imputer_poucos_na=SimpleImputer(strategy='most_frequent')


In [180]:
#Labels raras

rare_threshold=1/100
#Todas as labels que aparecem em proproção inferior a rare_thrshold serão agrupadas como uma única

#Identificar fearues que contem labels raras
rare_labels=[]
for feature in cat_features:
    if  not ((train[feature].value_counts()/len(train))>rare_threshold).all():
        rare_labels.append(feature)

rare_encoder = RareLabelEncoder(tol=rare_threshold, n_categories=1, variables=rare_labels)


In [181]:
#One hot encoder
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

#one hot encoding será aplicado somente a features com mais de duas labels
#e tambem aquelas com duas labels mas com muitos missing values, para a quais foi criada um label para indicar um missing value
columns_to_encode=[x for x in cat_features if train[x].nunique()>2 or x in cat_features_muitos_na]


In [182]:
#Converter para string para que tods os dados em uma mesma coluna tenham o mesmo formato
#Essa etapa é necessária para usar o one hot encoding

def converter_para_str(x):
    x=x.astype(str)
    return x

converter_str_transformer=FunctionTransformer(converter_para_str)

In [183]:
pipeline_cat_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na)),
            ('rare_labels',rare_encoder),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode))])

## Pipeline para dados com features selecionadas

De acordo com os resultados verificados com a regressões realizadas na etapa de análise exploratória, serão removidas as features que não tem associação com o output e as features que tenham correlação elevada com outra.

- Features numéricas discretas sem correlação com o output:
    - ps_calc_02
    - ps_calc_03
    - ps_calc_04
    - ps_calc_05
    - ps_calc_06
    - ps_calc_07
    - ps_calc_08
    - ps_calc_09
    - ps_calc_10
    - ps_calc_11
    - ps_calc_12
    - ps_calc_13
    - ps_calc_14

- Features categóricas não associadas com output:
    - ps_car_05_cat
    - ps_car_10_cat

- Variáveis binárias não associadas com output:
    - ps_ind_10_bin
    - ps_ind_11_bin
    - ps_ind_13_bin
    - ps_calc_15_bin
    - ps_calc_16_bin
    - ps_calc_17_bin
    - ps_calc_18_bin
    - ps_calc_19_bin
    - ps_calc_19_bin
    - ps_calc_20_bin



In [184]:
#features numéricas que serão removidas
remove_num_features=['ps_calc_02',
    'ps_calc_03',
    'ps_calc_04',
    'ps_calc_05',
    'ps_calc_06',
    'ps_calc_07',
    'ps_calc_08',
    'ps_calc_09',
    'ps_calc_10',
    'ps_calc_11',
    'ps_calc_12',
    'ps_calc_13',
    'ps_calc_14']

#features categóricas que serão removidas

remove_cat_features=['ps_car_05_cat','ps_car_10_cat']


#features binárias que serão removidas
remove_bin_features=[
    'ps_ind_10_bin',
    'ps_ind_11_bin',
    'ps_ind_13_bin',
    'ps_calc_15_bin',
    'ps_calc_16_bin',
    'ps_calc_17_bin',
    'ps_calc_18_bin',
    'ps_calc_19_bin',
    'ps_calc_19_bin',
    'ps_calc_20_bin'
]



### Features Binárias

In [185]:
#Não é necessário nenhuma alteração com relação a pipeline definida para os dados com todas as features
pipeline_bin_features=Pipeline([('replace_-1_nan',impute_nan_missing_value_transformer),
                          ('convert_to_object',converter_object_transformer),
                          ('fill_missing_binary',imputer_binary)])

In [186]:
pipeline_bin_features

Pipeline(steps=[('replace_-1_nan',
                 FunctionTransformer(func=<function impute_nan_missing_value at 0x7fe07f5ba4c0>)),
                ('convert_to_object',
                 FunctionTransformer(func=<function converter_object at 0x7fe07f5ba820>)),
                ('fill_missing_binary',
                 SimpleImputer(strategy='most_frequent'))])

### Features Numéricas

Será necessário alterar algumas das transformações aplicadas nos dados com todas as features, porque algumas das features as quais seria aplicadas as transformações foram removidas

In [187]:
#Verificar os missing values nas features numéricas removidas

train[remove_num_features].isna().sum()/len(train)

ps_calc_02    0.0
ps_calc_03    0.0
ps_calc_04    0.0
ps_calc_05    0.0
ps_calc_06    0.0
ps_calc_07    0.0
ps_calc_08    0.0
ps_calc_09    0.0
ps_calc_10    0.0
ps_calc_11    0.0
ps_calc_12    0.0
ps_calc_13    0.0
ps_calc_14    0.0
dtype: float64

In [188]:
#Missing values em todas as features numéricas
train[num_features].isna().sum()/len(train)

ps_ind_01     0.000000
ps_ind_03     0.000000
ps_ind_14     0.000000
ps_ind_15     0.000000
ps_reg_01     0.000000
ps_reg_02     0.000000
ps_reg_03     0.181065
ps_car_11     0.000008
ps_car_12     0.000002
ps_car_13     0.000000
ps_car_14     0.071605
ps_car_15     0.000000
ps_calc_01    0.000000
ps_calc_02    0.000000
ps_calc_03    0.000000
ps_calc_04    0.000000
ps_calc_05    0.000000
ps_calc_06    0.000000
ps_calc_07    0.000000
ps_calc_08    0.000000
ps_calc_09    0.000000
ps_calc_10    0.000000
ps_calc_11    0.000000
ps_calc_12    0.000000
ps_calc_13    0.000000
ps_calc_14    0.000000
dtype: float64

In [189]:
#não foi removida nenhuma features com muitos valores ausentes
remaining_num_features=[feature for feature in num_features if feature not in remove_num_features]

print('Número inicial de features numéricas: {}'.format(len(num_features)))
print('Número final de features numéricas após a remoção: {}'.format(len(remaining_num_features)))

Número inicial de features numéricas: 26
Número final de features numéricas após a remoção: 13


In [190]:
#numeric features com muitos mussing values
num_features_muitos_na_remaining=train[remaining_num_features].isna().mean()>missing_threshold
num_features_muitos_na_remaining=num_features_muitos_na_remaining[num_features_muitos_na_remaining].index.to_list()

#numeric features com poucos missing values
num_features_poucos_na_remaining=train[remaining_num_features].isna().mean()<missing_threshold
num_features_poucos_na_remaining=num_features_poucos_na_remaining[num_features_poucos_na_remaining].index.to_list()

In [191]:
#Não foi removida nenhuma feature numérica com muitos na
print('Número inicial de features numéricas com muitos na: {}'.format(len(num_features_muitos_na)))
print('Número final de features numéricas com muitos na: {}'.format(len(num_features_muitos_na_remaining)))


Número inicial de features numéricas com muitos na: 2
Número final de features numéricas com muitos na: 2


In [43]:
#adicionar colunas indicando valor ausente  
missing_indicator=AddMissingIndicator(variables=num_features_muitos_na)
#substituir valor ausente pela média
num_imputer=SimpleImputer(strategy='mean')


In [192]:
#Transformação de Yeo Jhonson
#As features que serão tranformadas permanecem as mesmas
features_to_be_transformed=['ps_reg_03',
                            'ps_car_12',
                            'ps_car_13',
                            'ps_car_14',
                            'ps_car_15',
                            'ps_reg_02']

features_to_be_transformed_remaining=[feature for feature in features_to_be_transformed if feature in remaining_num_features] 
print('Número inicial de features que receberão a transformação de Yeo Jhonson: {}'.format(len(features_to_be_transformed)))
print('Número final de features que receberão a transformação de Yeo Jhonson: {}'.format(len(features_to_be_transformed)))

Número inicial de features que receberão a transformação de Yeo Jhonson: 6
Número final de features que receberão a transformação de Yeo Jhonson: 6


In [ ]:
# Scaling

scaler = StandardScaler()

In [51]:
#Não é necessário fazer nenhuma alteração na pipeline para dados numéricos definida anteriormente
pipeline_num_features=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('addin_missing_indicator',missing_indicator),
            ('imputer_mean',SklearnTransformerWrapper(transformer=num_imputer,variables=num_features)),
            ('ajustar_ditribuicao',yeo_transformer),
            ('standard_scaler',SklearnTransformerWrapper(transformer=scaler,variables=num_features))])
pipeline_num_features

Pipeline(steps=[('replace_-1_nan',
                 FunctionTransformer(func=<function impute_nan_missing_value at 0x7fe0a8243430>)),
                ('addin_missing_indicator',
                 AddMissingIndicator(variables=['ps_reg_03', 'ps_car_14'])),
                ('imputer_mean',
                 SklearnTransformerWrapper(transformer=SimpleImputer(),
                                           variables=['ps_ind_01', 'ps_ind_03',
                                                      'ps_ind_14', 'ps_ind_15',
                                                      'ps_reg_01', 'ps_reg_02',...
                 SklearnTransformerWrapper(transformer=StandardScaler(),
                                           variables=['ps_ind_01', 'ps_ind_03',
                                                      'ps_ind_14', 'ps_ind_15',
                                                      'ps_reg_01', 'ps_reg_02',
                                                      'ps_reg_03', 'ps_car_11',
                                                      'ps_car_12', 'ps_car_13',
                                                      'ps_car_14', 'ps_car_15',
                                                      'ps_calc_01',
                                                      'ps_calc_02',
                                                      'ps_calc_03',
                                                      'ps_calc_04',
                                                      'ps_calc_05',
                                                      'ps_calc_06',
                                                      'ps_calc_07',
                                                      'ps_calc_08',
                                                      'ps_calc_09',
                                                      'ps_calc_10',
                                                      'ps_calc_11',
                                                      'ps_calc_12',
                                                      'ps_calc_13',
                                                      'ps_calc_14']))])

### Features Categóricas

In [193]:
#missing values na features categóricas
train[cat_features].isna().sum()/len(train)

ps_ind_02_cat    0.000363
ps_ind_04_cat    0.000139
ps_ind_05_cat    0.009760
ps_car_01_cat    0.000180
ps_car_02_cat    0.000008
ps_car_03_cat    0.690898
ps_car_04_cat    0.000000
ps_car_05_cat    0.447825
ps_car_06_cat    0.000000
ps_car_07_cat    0.019302
ps_car_08_cat    0.000000
ps_car_09_cat    0.000956
ps_car_10_cat    0.000000
ps_car_11_cat    0.000000
dtype: float64

In [194]:
#Missing values nas featues removidas
#ps_car_05_cat será mantida pq aparentemente a média do output é diferente comparando-se valores presentes e ausentes
train[remove_cat_features].isna().sum()/len(train)

ps_car_05_cat    0.447825
ps_car_10_cat    0.000000
dtype: float64

In [195]:
remove_cat_features=['ps_car_10_cat'] #somente ps_car_10_cat será removida
cat_features_remaining=[feature for feature in cat_features if feature not in remove_cat_features]

In [196]:
missing_threshold=1/100
#features categóricas com número de missing values superior ao thresold
cat_features_muitos_na_remaining=train[cat_features_remaining].isna().mean()>missing_threshold
cat_features_muitos_na_remaining=cat_features_muitos_na_remaining[cat_features_muitos_na_remaining].index.to_list()

#features categóricas com número de missing values inferior ao thresold
cat_features_poucos_na_remaining=train[cat_features_remaining].isna().mean()<missing_threshold
cat_features_poucos_na_remaining=cat_features_poucos_na_remaining[cat_features_poucos_na_remaining].index.to_list()

In [ ]:
#Substituir missing values nas features com muitos valores ausentes
cat_imputer_muitos_na=SimpleImputer(strategy='constant',fill_value='missing')

#Substituir missing values nas features com poucos valores ausentes
cat_imputer_poucos_na=SimpleImputer(strategy='most_frequent')

In [199]:
#Labels raras

rare_threshold=1/100
#Todas as labels que aparecem em proproção inferior a rare_thrshold serão agrupadas como uma única

#Identificar fearues que contem labels raras
rare_labels_remaining=[]
for feature in cat_features_remaining:
    if  not ((train[feature].value_counts()/len(train))>rare_threshold).all():
        rare_labels_remaining.append(feature)

rare_encoder_remaining = RareLabelEncoder(tol=rare_threshold, n_categories=1, variables=rare_labels_remaining)


In [ ]:
#One hot encoder
enc = OneHotEncoder(handle_unknown='ignore',sparse_output=False)

#one hot encoding será aplicado somente a features com mais de duas labels
#e tambem aquelas com duas labels mas com muitos missing values, para a quais foi criada um label para indicar um missing value
columns_to_encode_remaining=[x for x in cat_features_remaining if train[x].nunique()>2 or x in cat_features_muitos_na_remaining]


In [ ]:
#Converter para string para que tods os dados em uma mesma coluna tenham o mesmo formato
#Essa etapa é necessária para usar o one hot encoding

def converter_para_str(x):
    x=x.astype(str)
    return x

converter_str_transformer=FunctionTransformer(converter_para_str)

In [ ]:
pipeline_cat_features_remaining=Pipeline([
            ('replace_-1_nan',impute_nan_missing_value_transformer),
            ('converter_object',converter_object_transformer),
            ('imputer_muitos_na',SklearnTransformerWrapper(transformer=cat_imputer_muitos_na,variables=cat_features_muitos_na_remaining)),
            ('imputer_poucos_na',SklearnTransformerWrapper(transformer=cat_imputer_poucos_na,variables=cat_features_poucos_na_remaining)),
            ('rare_labels',rare_encoder),
            ('converter_str',SklearnTransformerWrapper(transformer=converter_str_transformer,variables=columns_to_encode_remaining)),
            ('one_hot',SklearnTransformerWrapper(transformer=enc,variables=columns_to_encode_remaining))])